# Vectorization and broadcasting

## $ \S 1 $ Vectorization


Suppose that we have three-dimensional arrays $ \mathbf u $ and $ \mathbf v $
and would like to take their inner (dot) product. Here's the most obvious way of doing
this in Python:

In [ ]:
import numpy as np

dimension = 3
u = np.array([1, 2, 3])
v = np.array([-1, 0, 1])

product = 0
for i in range(dimension):
    product += u[i] * v[i]
print(product)

2


This approach's limitation is that it performs the required arithmetical
operations *in sequence*. That is, we first compute the product $ 1 \cdot (-1) $
and add it to our cumulative total; then we add $ 2 \cdot 0 $; and finally we add
$ 3 \cdot 1 $ to arrive at the result.

We can speed up computations of this kind significantly by processing the
entire arrays, or at least large chunks of the arrays, at once. This technique
is called **vectorization**. It leverages the optimized implementations of vector and
matrix operations provided by libraries such as NumPy, which make use of
multi-core CPUs or even GPUs (Graphics Processing Units) to execute tasks *in
parallel* whenever possible. 

The vectorized version of the previous example would be:

In [ ]:
import numpy as np

u = np.array([1, 2, 3])
v = np.array([-1, 0, 1])

product = np.dot(u, v)  # Compute the dot product of u and v.
print(product)

2


This version is certainly more concise and legible. However, because the vectors
have only $ 3 $ dimensions, the performance boost is not noticeable. To better
appreciate the power of vectorization, let's consider the task of computing
the dot product of two vectors having $ 10 $ million dimensions. Here is the
vectorized version:

In [ ]:
import time  # Module that will allow us to time the computations

# Generate two large vectors with random coordinates in [0, 1):
dimension = 10**7
u = np.random.rand(dimension)  
v = np.random.rand(dimension)

tic = time.time()
prod = np.dot(u, v)
toc = time.time()
vect_runtime = toc - tic

print(f"The inner product is {prod}.")
print(f"Runtime for vectorized version: {1000 * vect_runtime} ms.")

The inner product is 2499843.426373514.
Runtime for vectorized version: 4.890680313110352 ms.


📝 The function `time.time()` used above returns a floating point number that
represents the time in seconds since January 1st, 1970, 00:00:00 (UTC). The
duration of the computation was measured by taking the time before (`tic`) and
after (`toc`) it, and then taking the difference. 

Considering the dimension of the vectors, the computation was pretty fast. Let's
now contrast the performance to that of its non-vectorized counterpart.

In [ ]:
import time  # A module that will allow us to time the computations

tic = time.time()
prod = 0
for i in range(dimension):
    prod += u[i] * v[i]
toc = time.time()
non_vect_runtime = toc - tic
print(f"The inner product is {prod}.")
print(f"Runtime for vectorized version: {non_vect_runtime} s.")
print(f"Or, in miliseconds: {1000 * (non_vect_runtime)}")

The inner product is 2499843.4263739116.
Runtime for vectorized version: 1.4287347793579102 s.
Or, in miliseconds: 1428.7347793579102


📝 The slight discrepancy in the value of the dot product between the two
versions arises because the non-vectorized method accumulates floating-point
arithmetic errors more prominently than the vectorized operation.

**Exercise:** Referring to the example above, compute the precise 
speedup factor relating the two implementations. Why does this value
change everytime you compute it?

This discussion shows that vectorization provides a simple way to improve the
performance of our code by several orders of magnitude. This is especially
crucial in machine learning and computer graphics. These fields frequently
involve dealing with vast datasets and complex numerical computations. In such
cases, code that does not make use of vectorization is simply not viable.

To summarize: *When performing operations on arrays, avoid loops whenever
possible; instead, use the built-in vectorized implementations provided by
NumPy.*

## $ \S 3 $ Vector- and matrix-valued functions

To illustrate another aspect of vectorization, let $ \mathbf u = (1, 2, 3) $ and
suppose that we need to apply the exponential function to each of the
coordinates of $ \mathbf u $. That is, suppose that we wish to compute $ \exp(u)
= (e^1, e^2, e^3) $. We could simply use a for-loop:

In [ ]:
u = np.array([1, 2, 3])
v = np.zeros(3)  # will hold the result
for i in range(3):
    v[i] = np.exp(u[i])
print(v)

[ 2.71828183  7.3890561  20.08553692]


However, there is an alternative way that is both simpler and more efficient,
namely, to use vectorization, in this case by leveraging to NumPy to apply the
exponential to the vector $ u $ as a whole:

In [ ]:
v = np.exp(u)
print(v)

[ 2.71828183  7.3890561  20.08553692]


This approach also works with basic operations that are built into Python, such as:

In [ ]:
squared_u = u**2
print(squared_u)

[1 4 9]


In [ ]:
reciprocal_u = 1 / u
print(reciprocal_u)

[1.         0.5        0.33333333]


__Exercise:__ Similarly to what we did for the inner product, compare the difference in performance
between the vectorized and non-vectorized approaches to computing $ e^u $, where $ u $ is a random
vector having a large number of dimensions. What is the speedup factor, approximately?

__Exercise:__ Let $ u = (1, 2, 3) $. Compute:

(a) $ \log(u) $ (that is, the vector whose coordinates are the logarithms of the coordinates of $ u $). *Hint*: Use the `np.log` function.

(b) $ \vert{u}\vert $ (that is, the vector whose coordinates are the absolute values of the coordinates of $ u $). *Hint:* Use the `np.abs` function.

(c) $ \max\{u, 3\} $. *Hint:* Use the `np.max` function.

(d) $ \sin\big(\tfrac{\pi u}{2}\big) $. *Hint:* Use the `np.sin` function.

How did you interpret the vectors of items (c) and (d)?

## $ \S 8 $ Basic terminology of probability theory

NumPy provides a comprehensive toolkit for random number generation.  However,
before discussing these tools, let us first briefly explain some basic
terminology from probability theory that are necessary to grasp what is going on
behind the scenes.

### $ 8.1 $ A simple example

Consider an experiment which consists of rolling a six-faced die. Then the set
of all possible _outcomes_ is given by $ \Omega = \{1, 2, \cdots, 6\} $.  This
is our _sample space_ in this case. An _event_, such as rolling an even number,
is simply a subset of $ \Omega $, in this case $ E = \{2, 4, 6\} $. Another
event would be rolling a face greater than $ 4 $, described by $ A = \{5, 6\} $.

If the die is perfectly balanced, so that each outcome is equally likely, then
the _probability distribution_ on our sample space is the function $ f \colon
\Omega \to [0, 1] $ that assigns the value $ \frac{1}{6} $ to each face.
Actually, this is a very special distribution called the _uniform distribution_.
However, in practice the die will never be perfectly balanced, so that some
outcomes (faces) may be more probable than others. It is the purpose of the
probability distribution to describe how probabilities are allocated among the
possible outcomes.

The preceding example illustrates a _finite_ sample space. However, many
interesting phenomena, such as measuring the luminosity of a star or the
height of an individual, are more adequately modeled by continuous sample spaces.
In such cases, the probability of any single outcome is zero. The probability
of an event $ A $ is obtained by _integrating_ the probability distribution over
$ A $.  The formal definitions below will cover both cases.

### ⚡ $ 8.2 $ Formal definitions

Let $ \Omega $ be an arbitrary set (not necessarily finite). An
element of $ \Omega $ will be called an __outcome__, and a subset of $ \Omega $
will be called an __event__.  A __probability function__ on $ \Omega $ is a real
function defined on the set of events of $ \Omega $ that satisfies the following
three properties (called the __Kolmogorov axioms__):

1. (_Non-negativity_) For any event $ A $ in the sample space $ S $, the
probability of $ A $ is non-negative; in symbols, $ P(A) \ge 0 $.

2. (_Unit Measure_) The probability of the entire sample space is 1, that is,
$ P(\Omega) = 1 $. This axiom implies that the probability that some outcome in
the sample space will occur is certain.

3. (_Countable Additivity_) For any countable or finite sequence of mutually
exclusive events $ A_1, A_2, A_3, \ldots $ (meaning no two events have any
outcomes in common), the probability of the union of these events is equal to
the sum of their individual probabilities. Symbolically, if $A_i \cap A_j =
\emptyset$ for $i \neq j$, then
$$ P\left(\bigcup_{i=1}^{\infty} A_i\right) = \sum_{i=1}^{\infty} P(A_i)\,. $$

Formally, a __sample space__ is a pair consisting of a set $ \Omega $ and
a probability function $ P $ as above. (Actually, our definition is not
entirely correct because in the case where $ \Omega $ is not
discrete, it may not be possible to define $ P $ consistently over the
set of _all_ subsets of $ \Omega $. Because of this, we only require that $ P $
be defined over a so-called $ \sigma $-algebra, but we will ignore
these technical difficulties here).

Usually a sample space is described not in terms of the probability function
$ P $ above (which is defined on a set of subsets of $ \Omega $), but in terms
of a simpler function $ f $ defined over $ \Omega $ itself, called a
__probability distribution__.

A __discrete__ (or countable) sample space is one whose outcomes $ \{x_1, x_2,
\cdots, x_n,\cdots \} $ are in one-to-one correspondence with set of natural
numbers $ \{1, 2, \cdots, \} $ or one of its subsets. In this case, $ P $ is
completely determined by a suitable assignment of probabilities to each of the
outcomes $ x_k $, by means of the distribution function $ f \colon \Omega \to
[0, 1] $. In this context, $ f $ is also called a _probability mass function_.
Then for an arbitrary event $ A $, we have
$$
P(A) = \sum_{x_k \in A} f(x_k)\qquad (A \subset \Omega).
$$
Note that this series is necessarily convergent because the total probability
is $ 1 $.

For a uncountable sample space $ \Omega \subset \mathbb R^n $, we can again
describe the probability function $ P $ by a simpler distribution function $ f
\colon \Omega \to [0, 1] $. However, this time we _integrate instead of summing_:
$$
P(A) = \int_A f(x)\,dx \qquad (A \subset \Omega).
$$
In this context $ f $ is also called a __probability density function__.
In most cases we assume that $ f $ is continuous, so that the integral above
exists for any reasonable event $ A $ of $ \Omega $.

## $ \S 9 $ Random number generation

A common way to create an array is by populating it randomly. This is useful for
example to initialize weights in a neural network; another common application is
to randomly shuffle a dataset.


In [ ]:
# To generate random floating-point numbers between 0 and 1, use `rand`:
random_vector = np.random.rand(5)  # 1D array of 5 random floats
random_matrix = np.random.rand(2, 3)  # 2x3 array of random floats

print(random_vector, '\n')
print(random_matrix)

[0.61408305 0.75204832 0.62026719 0.30220243 0.47913732] 

[[0.03751491 0.90405153 0.23338299]
 [0.99608328 0.78971066 0.50382658]]


The function `rand()` without arguments draws a random floating-point number in
$ [0.0, 1.0) $ according to a uniform distribution, meaning that the probability
that the result lies in an interval $ I \subset [0, 1) $ is exactly the length $
\vert{I}\vert $ of $ I $. More generally, the arguments of `rand` describe the
size of the resulting array of random numbers, all drawn from this same distribution.
In particular, note that every call to `rand` results in new numbers.

NumPy also provides functions to rearrange arrays randomly, which is especially useful in simulations and algorithms that require random sampling without replacement:

* Shuffle (`shuffle`): Modifies an array in-place by shuffling its contents.
* Permutation (`permutation`): Returns a new array that is a randomly permuted sequence of its input.